In [ ]:
import os
import sys

# Get the current working directory
current_dir = os.getcwd()

# Get the parent directory of `maps` (which is `src`)
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

# Add `src` to the module search path
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [ ]:
import folium.features
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from python_scripts.ihm.maps.mapUtils import *
from python_scripts.neighbours_criteria.simple_criteria import *
from python_scripts.neighbours_criteria.enhanced_criteria import *
from python_scripts.miscellaneaous.data_processing import extract_data
from python_scripts.graphs.graphs_creation import delaunay_graph
from python_scripts.city.city_utils import mean_distance_choice, city_detection_enhanced


In [ ]:
df = pd.read_csv("../../database/data.csv", sep=";", decimal=',')
providers = df['nom_op'].unique()
regions = df['nom_reg'].unique()
technologies = pd.Series(['site_2g', 'site_3g', 'site_4g', 'site_5g'])
ntechs = len(technologies)
nprovs = len(providers)

In [ ]:
# Create graph
df_extracted = extract_data(df, provider='Orange', techno='4g', region='Normandie') #Occitanie, Île-de-France
G, pos = delaunay_graph(df_extracted)

In [ ]:
# detected_cities = city_detection_enhanced(extract_data(df, provider='Orange', techno='4g')[['x', 'y']])
# cityness_proba = detected_cities['probas']

In [ ]:
mean_distances = mean_distance_to_NN(df_extracted[['x', 'y']], n_neighbours=3) # 3 to have more neighbours

In [ ]:
mean_distance_params = {
    ']0, 1] km': {'colour': '#030464', 'angle': 40, 'distance': 2},
    ']1, 2] km': {'colour': '#069AF3', 'angle': 30, 'distance': 5},
    ']2, 4] km': {'colour': '#02D4BB', 'angle': 25, 'distance': 10},
    ']4, inf] km': {'colour': '#0DBF75', 'angle': 15, 'distance': 15},
}

In [ ]:
G_dis = distance_criterion_enhanced(G, pos, params=mean_distance_params, mean_distance_to_NN=mean_distances)#, cityness_proba=cityness_proba)
G_ang = angle_criterion_enhanced(G, pos, params=mean_distance_params, mean_distance_to_NN=mean_distances)
G_qua = quadrant_criterion_enhanced(G, pos)
G_tot = distance_criterion_enhanced(G, pos, params=mean_distance_params, mean_distance_to_NN=mean_distances)#, cityness_proba=cityness_proba)
G_tot = angle_criterion_enhanced(G_tot, pos, params=mean_distance_params, mean_distance_to_NN=mean_distances)
G_tot = quadrant_criterion_enhanced(G_tot, pos)

In [ ]:
G_tot.edges(18226) # ptet créer une nouvelle colone dans le dataframe

In [ ]:
# Create a colormap
cmap = plt.cm.get_cmap('winter_r')  # 'cool' colormap for blue to gray
norm = mcolors.Normalize(vmin=0, vmax=1)

In [ ]:
def add_graph_edges(G_base: nx.Graph, G: nx.Graph, pos: dict, fg: folium.FeatureGroup, colour: str):
    for edge in G_base.edges:
        stations = []
        if(not(edge in G.edges)):
            stations.append(pos[edge[0]])
            stations.append(pos[edge[1]])

            folium.PolyLine(np.array(stations), color=colour, weight=2.5, opacity=1).add_to(fg)

In [ ]:
def addLegend(map, labelsToColors):
    legend_html = f'''
    <div style="position: fixed; 
                bottom: 50px; left: 50px; width: 200px; height: 120px; 
                border:2px solid grey; z-index:9999; font-size:14px;
                background-color: white;
                "><strong>Legend</strong> <br>
    '''
    for label in labelsToColors.keys():
        legend_html += f"\n&nbsp; <span style='color:{labelsToColors.get(label).get('colour')}'>&#9632;</span> {label} <br>\n"
    legend_html += '</div>'
    map.get_root().html.add_child(folium.Element(legend_html))

In [ ]:
map = folium.Map(location=np.mean(df_extracted[['latitude','longitude']], axis=0), zoom_start=8.5, tiles="OpenStreetMap")#"Cartodb Positron")
labels = folium.FeatureGroup("Labels", show=False).add_to(map)
edges = folium.FeatureGroup(f"Edges ({len(G.edges)})").add_to(map)
edges_dis = folium.FeatureGroup(f"Edges - distance ({len(G_dis.edges)})", show=False).add_to(map)
edges_ang = folium.FeatureGroup(f"Edges - angle ({len(G_ang.edges)})", show=False).add_to(map)
edges_qua = folium.FeatureGroup(f"Edges - quadrant ({len(G_qua.edges)})", show=False).add_to(map)
edges_tot = folium.FeatureGroup(f"Edges - total ({len(G_tot.edges)})").add_to(map)
points = folium.FeatureGroup(f"Points ({len(G)})").add_to(map)

for edge in G.edges:
    stations = []
    stations.append(pos[edge[0]])
    stations.append(pos[edge[1]])
    stations = np.array(stations)
    
    folium.PolyLine(stations, color="lightgray", weight=2.5, opacity=1).add_to(edges)
    folium.map.Marker(
        (stations[0]+stations[1])/2,
        icon=folium.features.DivIcon(
            icon_size=(250,36),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 7pt">{km_distance(stations[0], stations[1]):.2f} km</div>'
        )
    ).add_to(labels)

add_graph_edges(G, G_dis, pos, edges_dis, colour="red")
add_graph_edges(G, G_ang, pos, edges_ang, colour="orange")
add_graph_edges(G, G_qua, pos, edges_qua, colour="green")
add_graph_edges(G_tot, nx.Graph(), pos, edges_tot, colour="#AAA662")

for ind, latitude, longitude in df_extracted[['latitude', 'longitude']].itertuples():
    color = mean_distance_choice(ind, mean_distances, mean_distance_params, 'colour')
    points.add_child(folium.CircleMarker(location=[latitude, longitude], color=color, radius=5, popup=f"mean_dist={mean_distances.get(ind)}"))
    #\nproba={cityness_proba.get(ind)}
folium.LayerControl().add_to(map)

addLegend(map, mean_distance_params)

map.save("../../out/maps/enhanced_neighborhood_v1.html")